In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
image=cv2.imread('../input/natural-images/data/natural_images/cat/cat_0353.jpg')
plt.imshow(image,cmap="gray"),plt.axis("off")
plt.show()

Given an image, you want to output a simplified version.


*Thresholding is the process of setting pixels with intensity greater than some value to be white and less than the value to be black. A more advanced technique is adaptive thresholding, where the threshold value for a pixel is determined by the pixel intensities of its neighbors. This can be helpful when lighting conditions change over different regions in an image:
*


In [ ]:
image_grey=cv2.imread('../input/natural-images/data/natural_images/cat/cat_0353.jpg',cv2.IMREAD_GRAYSCALE)
max_output_value=255
neighbourhood_size=99
subtract_from_mean=10
image_binarized=cv2.adaptiveThreshold(image_grey,max_output_value,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,neighbourhood_size,subtract_from_mean)
plt.imshow(image_binarized,cmap="gray"),plt.axis("off")
plt.show()

* max_output_value===> the maximum intensity of the output pixel intensities
* cv2.ADAPTIVE_THRESH_GAUSSIAN_C ===> pixel’s threshold to be a weighted sum of the neighboring pixel intensities
* The last two parameters are the block size (the size of the neighborhood used to determine a pixel’s threshold) and a constant subtracted from the calculated threshold (used to manually fine-tune the threshold). 
* THRESH_BIANRY==>A binary threshold is a simple "either or" threshold, where the pixels are either 255 or 0. In many cases, this would be white or black, but we have left our image colored for now, so it may be colored still. The first parameter here is the image. The next parameter is the threshold, we are choosing 10. The next is the maximum value, which we're choosing as 255. Next and finally we have the type of threshold, which we've chosen as THRESH_BINARY. Normally, a threshold of 10 would be somewhat poor of a choice. We are choosing 10, because this is a low-light picture, so we choose a low number. Normally something about 125-150 would probably work best.

In [ ]:
img_flower=cv2.imread('../input/natural-images/data/natural_images/flower/flower_0449.jpg')
plt.imshow(img_flower,cmap="gray"),plt.axis("off")
plt.show()

Find edges of an image

In [ ]:
median_intensity=np.median(img_flower)
lower_threshold = int(max(0, (1.0 - 0.33) * median_intensity)) 
upper_threshold = int(min(255, (1.0 + 0.33) * median_intensity))

image_canny = cv2.Canny(img_flower, lower_threshold, upper_threshold)
plt.imshow(image_canny, cmap="gray"), plt.axis("off") 
plt.show()


DETECTING CORNERS

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
image_gray = np.float32(image_gray)

block_size = 2 
aperture = 29 
free_parameter = 0.04


detector_responses = cv2.cornerHarris(image_gray,block_size,  aperture, free_parameter)

detector_responses = cv2.dilate(detector_responses, None)
threshold = 0.02 
image[detector_responses > threshold * detector_responses.max()] = [255,255,255]

image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(image_gray, cmap="gray"), plt.axis("off") 
plt.show()

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

corners_to_detect = 10 
minimum_quality_score = 0.05 
minimum_distance = 25
corners = cv2.goodFeaturesToTrack(image_gray, corners_to_detect,  minimum_quality_score, minimum_distance)

corners = np.float32(corners)

for corner in corners:   
    x, y = corner[0]    
    cv2.circle(image, (x,y), 10, (255,255,255), -1)
   




In [ ]:
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(image_rgb, cmap='gray'), plt.axis("off") 
plt.show()


CREATING FEATURES OG MACHINE LEARNING

Convert image data to one-dimensional vector 

In [ ]:
image_10X10=cv2.resize(image_grey,(10,10))
image_10X10.flatten()

Images are presented as a grid of pixels. If an image is in grayscale, each pixel is presented by one value (i.e., pixel intensity: 1 if white, 0 if black). 

In [ ]:
plt.imshow(image_10X10, cmap="gray"), plt.axis("off") 
plt.show()


In [ ]:
image_color=cv2.imread('../input/natural-images/data/natural_images/cat/cat_0353.jpg',cv2.IMREAD_COLOR)

If the image is in color, instead of each pixel being represented by one value, it is represented by multiple values (most often three) representing the channels (red, green, blue, etc.) that blend to make the final color of that pixel. For this reason, if our 10 × 10 image is in color, we will have 300 feature values for each observation

In [ ]:
image_color_10X10=cv2.resize(image_color,(10,10))
image_color_10X10.flatten().shape